## spatial

In [1]:
size=224*2

In [ ]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

# def class_4_type(root):
#     label = ""
#     if "0" in root:
#         label = "0"
#     elif "1" in root:
#         label = "1"
#     elif "2" in root:
#         label = "2"
#     else:
#         label = "3"
#     return label

# def class_2_type(root):
#     label = ""
#     if "雙踝" in root:
#         label = "0"
#     elif "三踝" in root:
#         label = "1"
#     return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  
    # elif class_count == 4:
    #     class_type = class_4_type     

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================
    

    ## front(AP+Mortise) or side(Lateral) 標籤
    # =========================
    part = ""
    if image_dir.split("//")[-1] == "front":
        part = "AP+Mortise"
    else:
        part = "Lateral"
    # =========================

    ## load chosen model
    # =========================
    if chosen_model == 'resnet50':
        preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
        pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # elif chosen_model == 'resnet152':
    #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    elif chosen_model == 'vgg16':
        preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    elif chosen_model == 'vgg19':
        preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    elif chosen_model == 'mobilenet':
        preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    elif chosen_model == 'mobilenet_v2':
        preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    elif chosen_model == 'efficientnet':
        preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    elif chosen_model == 'inception_v3':
        preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
        pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    else:
        raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    print("-------Training " + part + "-------")
    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])

    ## test  image
    # =========================
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(size, size),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    # =========================

    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]
        # print("K-fold training set label distribution:\n", k_fold_train['Label'].value_counts(normalize=True))
        # print("K-fold validation set label distribution:\n", k_fold_val['Label'].value_counts(normalize=True))
        
        ## train, validation image
        # =========================
        # 關閉翻轉
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
        train_images = train_generator.flow_from_dataframe(
            dataframe=k_fold_train,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=True,
            seed=42
        )
        val_images = val_generator.flow_from_dataframe(
            dataframe=k_fold_val,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )
        # =========================
        
        ## load model
        # =========================
        pretrained_model = pretrained_model_chosen(
            input_shape=(size, size, 3),
            include_top=False,
            weights='imagenet',
            pooling='avg')

        pretrained_model.trainable = False

        inputs = pretrained_model.input
        x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
        x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

        outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
        model = tf.keras.Model(inputs, outputs)
        # print(model.summary())
        # =========================

        ## compile and evaluate
        # =========================
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        ## early stop 
        # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
        # model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
        
        ## no early stop
        model.fit(train_images, validation_data=val_images, epochs=30)

        results = model.evaluate(test_images,  verbose=0)
        # =========================

        ## print each results
        # =========================
        test_acc.append(np.round(results[1], 2))
        pred = model.predict(test_images)
        predicted_labels = np.argmax(pred, axis=1)
        f1 = f1_score(test_images.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\spatial"
# =========================

chosen_models = [
    'resnet50',
    # 'resnet152',
    'vgg16',
    'vgg19',
    'mobilenet',
    'mobilenet_v2',
    'efficientnet',
    'inception_v3'
]

results = []
print(len(chosen_models))
for i in range(len(chosen_models)):
    tmp = train_cross_validation(image_dir=path, class_count=3, chosen_model=chosen_models[i])
    results.append(tmp)

for i in range(len(chosen_models)):
    print(f'----------------------Model {chosen_models[i]}----------------------')
    print(f"acc mean = {results[i]['acc_mean']}, std = {results[i]['acc_std']}, test_acc = {results[i]['test_acc']}")
    print(f"f1  mean = {results[i]['f1_mean']}, std = {results[i]['f1_std']}, test_f1 = {results[i]['test_f1']}")

7
-------Training Lateral-------
Found 80 validated image filenames belonging to 3 classes.
Found 252 validated image filenames belonging to 3 classes.
Found 64 validated image filenames belonging to 3 classes.
Epoch 1/30
4/4 [==============================] - 8s 1s/step - loss: 1.0324 - accuracy: 0.5079 - val_loss: 1.0098 - val_accuracy: 0.4062
Epoch 2/30
4/4 [==============================] - 2s 461ms/step - loss: 0.9558 - accuracy: 0.5357 - val_loss: 0.9390 - val_accuracy: 0.5625
Epoch 3/30
4/4 [==============================] - 2s 459ms/step - loss: 0.8842 - accuracy: 0.6746 - val_loss: 0.8894 - val_accuracy: 0.6562
Epoch 4/30
4/4 [==============================] - 2s 460ms/step - loss: 0.8247 - accuracy: 0.7183 - val_loss: 0.8291 - val_accuracy: 0.6562
Epoch 5/30
4/4 [==============================] - 2s 452ms/step - loss: 0.7692 - accuracy: 0.7222 - val_loss: 0.7804 - val_accuracy: 0.7031
Epoch 6/30
4/4 [==============================] - 2s 456ms/step - loss: 0.7204 - accuracy: 0

In [3]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

# def class_4_type(root):
#     label = ""
#     if "0" in root:
#         label = "0"
#     elif "1" in root:
#         label = "1"
#     elif "2" in root:
#         label = "2"
#     else:
#         label = "3"
#     return label

# def class_2_type(root):
#     label = ""
#     if "雙踝" in root:
#         label = "0"
#     elif "三踝" in root:
#         label = "1"
#     return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  
    # elif class_count == 4:
    #     class_type = class_4_type     

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================
    

    ## front(AP+Mortise) or side(Lateral) 標籤
    # =========================
    part = ""
    if image_dir.split("//")[-1] == "front":
        part = "AP+Mortise"
    else:
        part = "Lateral"
    # =========================

    ## load chosen model
    # =========================
    if chosen_model == 'resnet50':
        preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
        pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # elif chosen_model == 'resnet152':
    #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    elif chosen_model == 'vgg16':
        preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    elif chosen_model == 'vgg19':
        preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    elif chosen_model == 'mobilenet':
        preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    elif chosen_model == 'mobilenet_v2':
        preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    elif chosen_model == 'efficientnet':
        preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    elif chosen_model == 'inception_v3':
        preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
        pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    else:
        raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    print("-------Training " + part + "-------")
    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])

    ## test  image
    # =========================
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(size, size),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    # =========================

    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]
        # print("K-fold training set label distribution:\n", k_fold_train['Label'].value_counts(normalize=True))
        # print("K-fold validation set label distribution:\n", k_fold_val['Label'].value_counts(normalize=True))
        
        ## train, validation image
        # =========================
        # 關閉翻轉
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
        train_images = train_generator.flow_from_dataframe(
            dataframe=k_fold_train,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=True,
            seed=42
        )
        val_images = val_generator.flow_from_dataframe(
            dataframe=k_fold_val,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )
        # =========================
        
        ## load model
        # =========================
        if maru_part is not None:
            pretrained_model = tf.keras.models.load_model(maru_part)
        else:
            pretrained_model = pretrained_model_chosen(
                input_shape=(size, size, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

        pretrained_model.trainable = False

        inputs = pretrained_model.input
        x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
        x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

        outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
        model = tf.keras.Model(inputs, outputs)
        # print(model.summary())
        # =========================

        ## compile and evaluate
        # =========================
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        ## early stop 
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
        model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
        
        ## no early stop
        # model.fit(train_images, validation_data=val_images, epochs=30)

        results = model.evaluate(test_images,  verbose=0)
        # =========================

        ## print each results
        # =========================
        test_acc.append(np.round(results[1], 2))
        pred = model.predict(test_images)
        predicted_labels = np.argmax(pred, axis=1)
        f1 = f1_score(test_images.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\spatial"
# =========================

chosen_models = [
    'resnet50',
    # 'resnet152',
    'vgg16',
    'vgg19',
    'mobilenet',
    'mobilenet_v2',
    'efficientnet',
    'inception_v3'
]

results = []
print(len(chosen_models))
for i in range(len(chosen_models)):
    tmp = train_cross_validation(image_dir=path, class_count=3, chosen_model=chosen_models[i])
    results.append(tmp)

for i in range(len(chosen_models)):
    print(f'----------------------Model {chosen_models[i]}----------------------')
    print(f"acc mean = {results[i]['acc_mean']}, std = {results[i]['acc_std']}, test_acc = {results[i]['test_acc']}")
    print(f"f1  mean = {results[i]['f1_mean']}, std = {results[i]['f1_std']}, test_f1 = {results[i]['test_f1']}")

7
-------Training Lateral-------
Found 80 validated image filenames belonging to 3 classes.
Found 252 validated image filenames belonging to 3 classes.
Found 64 validated image filenames belonging to 3 classes.
Epoch 1/30
4/4 [==============================] - 4s 612ms/step - loss: 1.2193 - accuracy: 0.3095 - val_loss: 1.0623 - val_accuracy: 0.4688
Epoch 2/30
4/4 [==============================] - 2s 423ms/step - loss: 1.0579 - accuracy: 0.4048 - val_loss: 1.0351 - val_accuracy: 0.4062
Epoch 3/30
4/4 [==============================] - 2s 421ms/step - loss: 1.0085 - accuracy: 0.4643 - val_loss: 0.9837 - val_accuracy: 0.5938
Epoch 4/30
4/4 [==============================] - 2s 427ms/step - loss: 0.9545 - accuracy: 0.6468 - val_loss: 0.9389 - val_accuracy: 0.6719
Epoch 5/30
4/4 [==============================] - 2s 421ms/step - loss: 0.9119 - accuracy: 0.6627 - val_loss: 0.8939 - val_accuracy: 0.6562
Epoch 6/30
4/4 [==============================] - 2s 423ms/step - loss: 0.8685 - accuracy

In [4]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

# def class_4_type(root):
#     label = ""
#     if "0" in root:
#         label = "0"
#     elif "1" in root:
#         label = "1"
#     elif "2" in root:
#         label = "2"
#     else:
#         label = "3"
#     return label

# def class_2_type(root):
#     label = ""
#     if "雙踝" in root:
#         label = "0"
#     elif "三踝" in root:
#         label = "1"
#     return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  
    # elif class_count == 4:
    #     class_type = class_4_type     

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================
    

    ## front(AP+Mortise) or side(Lateral) 標籤
    # =========================
    part = ""
    if image_dir.split("//")[-1] == "front":
        part = "AP+Mortise"
    else:
        part = "Lateral"
    # =========================

    ## load chosen model
    # =========================
    if chosen_model == 'resnet50':
        preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
        pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # elif chosen_model == 'resnet152':
    #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    elif chosen_model == 'vgg16':
        preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    elif chosen_model == 'vgg19':
        preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    elif chosen_model == 'mobilenet':
        preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    elif chosen_model == 'mobilenet_v2':
        preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    elif chosen_model == 'efficientnet':
        preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    elif chosen_model == 'inception_v3':
        preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
        pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    else:
        raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    print("-------Training " + part + "-------")
    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])

    ## test  image
    # =========================
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(size, size),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    # =========================

    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]
        # print("K-fold training set label distribution:\n", k_fold_train['Label'].value_counts(normalize=True))
        # print("K-fold validation set label distribution:\n", k_fold_val['Label'].value_counts(normalize=True))
        
        ## train, validation image
        # =========================
        # 關閉翻轉
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
        train_images = train_generator.flow_from_dataframe(
            dataframe=k_fold_train,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=True,
            seed=42
        )
        val_images = val_generator.flow_from_dataframe(
            dataframe=k_fold_val,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )
        # =========================
        
        ## load model
        # =========================
        if maru_part is not None:
            pretrained_model = tf.keras.models.load_model(maru_part)
        else:
            pretrained_model = pretrained_model_chosen(
                input_shape=(size, size, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

        pretrained_model.trainable = False

        inputs = pretrained_model.input
        x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
        x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

        outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
        model = tf.keras.Model(inputs, outputs)
        # print(model.summary())
        # =========================

        ## compile and evaluate
        # =========================
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        ## early stop 
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
        model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
        
        ## no early stop
        # model.fit(train_images, validation_data=val_images, epochs=30)

        results = model.evaluate(test_images,  verbose=0)
        # =========================

        ## print each results
        # =========================
        test_acc.append(np.round(results[1], 2))
        pred = model.predict(test_images)
        predicted_labels = np.argmax(pred, axis=1)
        f1 = f1_score(test_images.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\spatial"
# =========================

chosen_models = [
    'resnet50',
    # 'resnet152',
    'vgg16',
    'vgg19',
    'mobilenet',
    'mobilenet_v2',
    'efficientnet',
    'inception_v3'
]

results = []
print(len(chosen_models))
for i in range(len(chosen_models)):
    tmp = train_cross_validation(image_dir=path, class_count=3, chosen_model=chosen_models[i])
    results.append(tmp)

for i in range(len(chosen_models)):
    print(f'----------------------Model {chosen_models[i]}----------------------')
    print(f"acc mean = {results[i]['acc_mean']}, std = {results[i]['acc_std']}, test_acc = {results[i]['test_acc']}")
    print(f"f1  mean = {results[i]['f1_mean']}, std = {results[i]['f1_std']}, test_f1 = {results[i]['test_f1']}")

7
-------Training Lateral-------
Found 80 validated image filenames belonging to 3 classes.
Found 252 validated image filenames belonging to 3 classes.
Found 64 validated image filenames belonging to 3 classes.
Epoch 1/30
4/4 [==============================] - 4s 617ms/step - loss: 1.2254 - accuracy: 0.3968 - val_loss: 1.0599 - val_accuracy: 0.4531
Epoch 2/30
4/4 [==============================] - 2s 426ms/step - loss: 1.0720 - accuracy: 0.4365 - val_loss: 1.0338 - val_accuracy: 0.4688
Epoch 3/30
4/4 [==============================] - 2s 427ms/step - loss: 1.0231 - accuracy: 0.5595 - val_loss: 0.9905 - val_accuracy: 0.5625
Epoch 4/30
4/4 [==============================] - 2s 423ms/step - loss: 0.9726 - accuracy: 0.6071 - val_loss: 0.9400 - val_accuracy: 0.6094
Epoch 5/30
4/4 [==============================] - 2s 429ms/step - loss: 0.9169 - accuracy: 0.6389 - val_loss: 0.9005 - val_accuracy: 0.6406
Epoch 6/30
4/4 [==============================] - 2s 423ms/step - loss: 0.8776 - accuracy

## channel

In [5]:
size=224

In [6]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

# def class_4_type(root):
#     label = ""
#     if "0" in root:
#         label = "0"
#     elif "1" in root:
#         label = "1"
#     elif "2" in root:
#         label = "2"
#     else:
#         label = "3"
#     return label

# def class_2_type(root):
#     label = ""
#     if "雙踝" in root:
#         label = "0"
#     elif "三踝" in root:
#         label = "1"
#     return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  
    # elif class_count == 4:
    #     class_type = class_4_type     

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================
    

    ## front(AP+Mortise) or side(Lateral) 標籤
    # =========================
    part = ""
    if image_dir.split("//")[-1] == "front":
        part = "AP+Mortise"
    else:
        part = "Lateral"
    # =========================

    ## load chosen model
    # =========================
    if chosen_model == 'resnet50':
        preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
        pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # elif chosen_model == 'resnet152':
    #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    elif chosen_model == 'vgg16':
        preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    elif chosen_model == 'vgg19':
        preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    elif chosen_model == 'mobilenet':
        preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    elif chosen_model == 'mobilenet_v2':
        preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    elif chosen_model == 'efficientnet':
        preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    elif chosen_model == 'inception_v3':
        preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
        pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    else:
        raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    print("-------Training " + part + "-------")
    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])

    ## test  image
    # =========================
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(size, size),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    # =========================

    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]
        # print("K-fold training set label distribution:\n", k_fold_train['Label'].value_counts(normalize=True))
        # print("K-fold validation set label distribution:\n", k_fold_val['Label'].value_counts(normalize=True))
        
        ## train, validation image
        # =========================
        # 關閉翻轉
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
        train_images = train_generator.flow_from_dataframe(
            dataframe=k_fold_train,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=True,
            seed=42
        )
        val_images = val_generator.flow_from_dataframe(
            dataframe=k_fold_val,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )
        # =========================
        
        ## load model
        # =========================
        if maru_part is not None:
            pretrained_model = tf.keras.models.load_model(maru_part)
        else:
            pretrained_model = pretrained_model_chosen(
                input_shape=(size, size, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

        pretrained_model.trainable = False

        inputs = pretrained_model.input
        x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
        x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

        outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
        model = tf.keras.Model(inputs, outputs)
        # print(model.summary())
        # =========================

        ## compile and evaluate
        # =========================
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        ## early stop 
        # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
        # model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
        
        ## no early stop
        model.fit(train_images, validation_data=val_images, epochs=30)

        results = model.evaluate(test_images,  verbose=0)
        # =========================

        ## print each results
        # =========================
        test_acc.append(np.round(results[1], 2))
        pred = model.predict(test_images)
        predicted_labels = np.argmax(pred, axis=1)
        f1 = f1_score(test_images.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\channel"
# =========================

chosen_models = [
    'resnet50',
    # 'resnet152',
    'vgg16',
    'vgg19',
    'mobilenet',
    'mobilenet_v2',
    'efficientnet',
    'inception_v3'
]

results = []
print(len(chosen_models))
for i in range(len(chosen_models)):
    tmp = train_cross_validation(image_dir=path, class_count=3, chosen_model=chosen_models[i])
    results.append(tmp)

for i in range(len(chosen_models)):
    print(f'----------------------Model {chosen_models[i]}----------------------')
    print(f"acc mean = {results[i]['acc_mean']}, std = {results[i]['acc_std']}, test_acc = {results[i]['test_acc']}")
    print(f"f1  mean = {results[i]['f1_mean']}, std = {results[i]['f1_std']}, test_f1 = {results[i]['test_f1']}")

7
-------Training Lateral-------
Found 80 validated image filenames belonging to 3 classes.
Found 252 validated image filenames belonging to 3 classes.
Found 64 validated image filenames belonging to 3 classes.
Epoch 1/30
4/4 [==============================] - 4s 549ms/step - loss: 1.3492 - accuracy: 0.3056 - val_loss: 1.1183 - val_accuracy: 0.3594
Epoch 2/30
4/4 [==============================] - 0s 120ms/step - loss: 1.0891 - accuracy: 0.4167 - val_loss: 1.0421 - val_accuracy: 0.4688
Epoch 3/30
4/4 [==============================] - 0s 118ms/step - loss: 1.0016 - accuracy: 0.4881 - val_loss: 0.9635 - val_accuracy: 0.5469
Epoch 4/30
4/4 [==============================] - 0s 119ms/step - loss: 0.9092 - accuracy: 0.5913 - val_loss: 0.8888 - val_accuracy: 0.5781
Epoch 5/30
4/4 [==============================] - 0s 119ms/step - loss: 0.8360 - accuracy: 0.7222 - val_loss: 0.8290 - val_accuracy: 0.6406
Epoch 6/30
4/4 [==============================] - 0s 121ms/step - loss: 0.7849 - accuracy

In [7]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

# def class_4_type(root):
#     label = ""
#     if "0" in root:
#         label = "0"
#     elif "1" in root:
#         label = "1"
#     elif "2" in root:
#         label = "2"
#     else:
#         label = "3"
#     return label

# def class_2_type(root):
#     label = ""
#     if "雙踝" in root:
#         label = "0"
#     elif "三踝" in root:
#         label = "1"
#     return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  
    # elif class_count == 4:
    #     class_type = class_4_type     

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================
    

    ## front(AP+Mortise) or side(Lateral) 標籤
    # =========================
    part = ""
    if image_dir.split("//")[-1] == "front":
        part = "AP+Mortise"
    else:
        part = "Lateral"
    # =========================

    ## load chosen model
    # =========================
    if chosen_model == 'resnet50':
        preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
        pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # elif chosen_model == 'resnet152':
    #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    elif chosen_model == 'vgg16':
        preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    elif chosen_model == 'vgg19':
        preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    elif chosen_model == 'mobilenet':
        preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    elif chosen_model == 'mobilenet_v2':
        preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    elif chosen_model == 'efficientnet':
        preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    elif chosen_model == 'inception_v3':
        preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
        pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    else:
        raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    print("-------Training " + part + "-------")
    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])

    ## test  image
    # =========================
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(size, size),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    # =========================

    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]
        # print("K-fold training set label distribution:\n", k_fold_train['Label'].value_counts(normalize=True))
        # print("K-fold validation set label distribution:\n", k_fold_val['Label'].value_counts(normalize=True))
        
        ## train, validation image
        # =========================
        # 關閉翻轉
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
        train_images = train_generator.flow_from_dataframe(
            dataframe=k_fold_train,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=True,
            seed=42
        )
        val_images = val_generator.flow_from_dataframe(
            dataframe=k_fold_val,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )
        # =========================
        
        ## load model
        # =========================
        if maru_part is not None:
            pretrained_model = tf.keras.models.load_model(maru_part)
        else:
            pretrained_model = pretrained_model_chosen(
                input_shape=(size, size, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

        pretrained_model.trainable = False

        inputs = pretrained_model.input
        x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
        x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

        outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
        model = tf.keras.Model(inputs, outputs)
        # print(model.summary())
        # =========================

        ## compile and evaluate
        # =========================
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        ## early stop 
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
        model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
        
        ## no early stop
        # model.fit(train_images, validation_data=val_images, epochs=30)

        results = model.evaluate(test_images,  verbose=0)
        # =========================

        ## print each results
        # =========================
        test_acc.append(np.round(results[1], 2))
        pred = model.predict(test_images)
        predicted_labels = np.argmax(pred, axis=1)
        f1 = f1_score(test_images.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\channel"
# =========================

chosen_models = [
    'resnet50',
    # 'resnet152',
    'vgg16',
    'vgg19',
    'mobilenet',
    'mobilenet_v2',
    'efficientnet',
    'inception_v3'
]

results = []
print(len(chosen_models))
for i in range(len(chosen_models)):
    tmp = train_cross_validation(image_dir=path, class_count=3, chosen_model=chosen_models[i])
    results.append(tmp)

for i in range(len(chosen_models)):
    print(f'----------------------Model {chosen_models[i]}----------------------')
    print(f"acc mean = {results[i]['acc_mean']}, std = {results[i]['acc_std']}, test_acc = {results[i]['test_acc']}")
    print(f"f1  mean = {results[i]['f1_mean']}, std = {results[i]['f1_std']}, test_f1 = {results[i]['test_f1']}")

7
-------Training Lateral-------
Found 80 validated image filenames belonging to 3 classes.
Found 252 validated image filenames belonging to 3 classes.
Found 64 validated image filenames belonging to 3 classes.
Epoch 1/30
4/4 [==============================] - 2s 294ms/step - loss: 1.0991 - accuracy: 0.3730 - val_loss: 1.0393 - val_accuracy: 0.4531
Epoch 2/30
4/4 [==============================] - 1s 122ms/step - loss: 0.9796 - accuracy: 0.5437 - val_loss: 0.9427 - val_accuracy: 0.5781
Epoch 3/30
4/4 [==============================] - 1s 127ms/step - loss: 0.8868 - accuracy: 0.6190 - val_loss: 0.8623 - val_accuracy: 0.6719
Epoch 4/30
4/4 [==============================] - 1s 121ms/step - loss: 0.8126 - accuracy: 0.6905 - val_loss: 0.8082 - val_accuracy: 0.6875
Epoch 5/30
4/4 [==============================] - 1s 125ms/step - loss: 0.7493 - accuracy: 0.7183 - val_loss: 0.7687 - val_accuracy: 0.6875
Epoch 6/30
4/4 [==============================] - 1s 122ms/step - loss: 0.7006 - accuracy

In [8]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

# def class_4_type(root):
#     label = ""
#     if "0" in root:
#         label = "0"
#     elif "1" in root:
#         label = "1"
#     elif "2" in root:
#         label = "2"
#     else:
#         label = "3"
#     return label

# def class_2_type(root):
#     label = ""
#     if "雙踝" in root:
#         label = "0"
#     elif "三踝" in root:
#         label = "1"
#     return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  
    # elif class_count == 4:
    #     class_type = class_4_type     

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================
    

    ## front(AP+Mortise) or side(Lateral) 標籤
    # =========================
    part = ""
    if image_dir.split("//")[-1] == "front":
        part = "AP+Mortise"
    else:
        part = "Lateral"
    # =========================

    ## load chosen model
    # =========================
    if chosen_model == 'resnet50':
        preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
        pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # elif chosen_model == 'resnet152':
    #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    elif chosen_model == 'vgg16':
        preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    elif chosen_model == 'vgg19':
        preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
        pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    elif chosen_model == 'mobilenet':
        preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    elif chosen_model == 'mobilenet_v2':
        preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
        pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    elif chosen_model == 'efficientnet':
        preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
        pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    elif chosen_model == 'inception_v3':
        preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
        pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    else:
        raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    print("-------Training " + part + "-------")
    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])

    ## test  image
    # =========================
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(size, size),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    # =========================

    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]
        # print("K-fold training set label distribution:\n", k_fold_train['Label'].value_counts(normalize=True))
        # print("K-fold validation set label distribution:\n", k_fold_val['Label'].value_counts(normalize=True))
        
        ## train, validation image
        # =========================
        # 關閉翻轉
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
        train_images = train_generator.flow_from_dataframe(
            dataframe=k_fold_train,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=True,
            seed=42
        )
        val_images = val_generator.flow_from_dataframe(
            dataframe=k_fold_val,
            x_col='Filepath',
            y_col='Label',
            target_size=(size, size),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )
        # =========================
        
        ## load model
        # =========================
        if maru_part is not None:
            pretrained_model = tf.keras.models.load_model(maru_part)
        else:
            pretrained_model = pretrained_model_chosen(
                input_shape=(size, size, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

        pretrained_model.trainable = False

        inputs = pretrained_model.input
        x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
        x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

        outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
        model = tf.keras.Model(inputs, outputs)
        # print(model.summary())
        # =========================

        ## compile and evaluate
        # =========================
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        ## early stop 
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
        model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
        
        ## no early stop
        # model.fit(train_images, validation_data=val_images, epochs=30)

        results = model.evaluate(test_images,  verbose=0)
        # =========================

        ## print each results
        # =========================
        test_acc.append(np.round(results[1], 2))
        pred = model.predict(test_images)
        predicted_labels = np.argmax(pred, axis=1)
        f1 = f1_score(test_images.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\channel"
# =========================

chosen_models = [
    'resnet50',
    # 'resnet152',
    'vgg16',
    'vgg19',
    'mobilenet',
    'mobilenet_v2',
    'efficientnet',
    'inception_v3'
]

results = []
print(len(chosen_models))
for i in range(len(chosen_models)):
    tmp = train_cross_validation(image_dir=path, class_count=3, chosen_model=chosen_models[i])
    results.append(tmp)

for i in range(len(chosen_models)):
    print(f'----------------------Model {chosen_models[i]}----------------------')
    print(f"acc mean = {results[i]['acc_mean']}, std = {results[i]['acc_std']}, test_acc = {results[i]['test_acc']}")
    print(f"f1  mean = {results[i]['f1_mean']}, std = {results[i]['f1_std']}, test_f1 = {results[i]['test_f1']}")

7
-------Training Lateral-------
Found 80 validated image filenames belonging to 3 classes.
Found 252 validated image filenames belonging to 3 classes.
Found 64 validated image filenames belonging to 3 classes.
Epoch 1/30
4/4 [==============================] - 2s 306ms/step - loss: 1.2602 - accuracy: 0.3175 - val_loss: 1.0304 - val_accuracy: 0.3594
Epoch 2/30
4/4 [==============================] - 1s 121ms/step - loss: 0.9729 - accuracy: 0.5397 - val_loss: 0.9376 - val_accuracy: 0.5938
Epoch 3/30
4/4 [==============================] - 0s 120ms/step - loss: 0.8991 - accuracy: 0.6071 - val_loss: 0.9080 - val_accuracy: 0.5938
Epoch 4/30
4/4 [==============================] - 1s 121ms/step - loss: 0.8521 - accuracy: 0.6151 - val_loss: 0.8493 - val_accuracy: 0.6250
Epoch 5/30
4/4 [==============================] - 0s 120ms/step - loss: 0.7823 - accuracy: 0.6548 - val_loss: 0.7933 - val_accuracy: 0.6250
Epoch 6/30
4/4 [==============================] - 1s 124ms/step - loss: 0.7340 - accuracy